<a href="https://colab.research.google.com/github/sayanbanerjee32/TASI_ERAv2_S20/blob/main/bengali_bpe_tokenizer_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# copy the begali corpus file
!cp /content/drive/MyDrive/ERAv2_datasets/bengali_corpus/ben.txt .

In [3]:
with open('ben.txt', 'r') as _f:
    # raw_text = _f.readlines()
    raw_lines = [line.split('\t')[1] for line in _f]
    ben_text = '\n'.join(raw_lines)

ben_text[-1000:]

'্য সবাই যা করতে চায় তা করতে চায়নি।\nএকা থাকার একটা অসুবিধে হোলো যে কথা বলার মতো কেউ থাকে না।\nআপনার সমস্ত অর্থ দিয়ে আপনি যা চান তা কিনতে সক্ষম হওয়া উচিত।\nবাইরে এতই গরম যে আমি পুরোদিন আমার শীততাপ নিয়ন্ত্রিত বাড়িতে থাকতে চাই।\nসকালে সূর্য পূর্ব দিকে ওঠে এবং সন্ধ্যায় পশ্চিমে অস্ত যায়।\nআমি শুনলাম যে তাঁরা পার্ক স্ট্রিটের একটা বাড়ির ভিতের মধ্যে থেকে একটা কঙ্কাল পেয়েছেন।\nটম যখন শুনলো যে মেরি আরে জন বিয়ে করেছে তখন তাকে দেখে বেশ অবাক মনে হয়েছিলো।\nআমার মনে হয় টম যে সোনা পেয়েছে সেটা তার কাছে রাখতে দেওয়া হবে এমন সম্ভাবনা খুব কম।\nটম মেরিকে বললো যে ও নিজেকে হত্যা করতে চলেছিলো, কিন্ত তা করার মতো সাহস ছিলো না।\nটমের সঙ্গে কাজ করা খুব বিরক্তিকর কারণ ও কখনই মেনে নেয় না যে ও ভুল করেছে।\nআমি ভেবেছিলাম এটা করা সহজ হবে, কিন্তু আমরা সারাদিন ধরে কাজ করেছি আর এখনো শেষ করে উঠতে পারিনি।\nআমি ভেবেছিলাম এটা করা সহজ হবে, কিন্তু আমরা সারাদিন ধরে কাজ করেছি আর এখনো শেষ করে উঠতে পারিনি।\nবছরের বারোটা মাস হলো জানুয়ারি, ফেব্রুয়ারি, মার্চ, এপ্রিল, মে, জুন জুলাই, আগস্ট, সেপ্টেম্বর, অক্টোবর, নভেম্বর

In [14]:
tokens = ben_text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience
print('---')
print(ben_text[:100])
print("length:", len(ben_text))
print('---')
print(tokens[:100])
print("length:", len(tokens))

---
যাও।
যান।
যা।
পালাও!
পালান!
কে?
বাহ!
আগুন!
বাঁচাও!
বাঁচান!
ঝাঁপ দাও।
থামুন!
থামো!
থাম!
নমস্কার!
হ্যা
length: 169112
---
[224, 166, 175, 224, 166, 190, 224, 166, 147, 224, 165, 164, 10, 224, 166, 175, 224, 166, 190, 224, 166, 168, 224, 165, 164, 10, 224, 166, 175, 224, 166, 190, 224, 165, 164, 10, 224, 166, 170, 224, 166, 190, 224, 166, 178, 224, 166, 190, 224, 166, 147, 33, 10, 224, 166, 170, 224, 166, 190, 224, 166, 178, 224, 166, 190, 224, 166, 168, 33, 10, 224, 166, 149, 224, 167, 135, 63, 10, 224, 166, 172, 224, 166, 190, 224, 166, 185, 33, 10, 224, 166, 134, 224, 166, 151, 224, 167, 129, 224, 166]
length: 443302


In [ ]:
ord('য'), chr(2479), ord("!"), chr(33)

(2479, 'য', 33, '!')

In [ ]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]): # Pythonic way to iterate consecutive elements
        counts[pair] = counts.get(pair, 0) + 1
    return counts

stats = get_stats(tokens)

In [ ]:
top_pair = max(stats, key=stats.get)
top_pair, chr(top_pair[0]), chr(top_pair[1])

((224, 166), 'à', '¦')

In [ ]:
tokens = ben_text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience
len(tokens), len(set(tokens))

(443302, 74)

In [ ]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

# ---
vocab_size = 1001 # the desired final vocabulary size
num_merges = vocab_size - len(set(tokens))
ids = list(tokens) # copy so we don't destroy the original list

merges = {} # (int, int) -> int
for i in range(num_merges):
  stats = get_stats(ids)
  pair = max(stats, key=stats.get)
  idx = 256 + i
#   print(f"merging {pair} into a new token {idx}")
  ids = merge(ids, pair, idx)
  merges[pair] = idx
  if stats[pair] <= 2:
    break

print(f"vocab size: {len(set(ids))}")
print(f"Number of merges: {i}")
print(f"Number of occurences of last merged tokens: {stats[pair]}")

vocab size: 986
Number of merges: 926
Number of occurences of last merged tokens: 19


In [ ]:
print("tokens length:", len(tokens))
print("ids length:", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

tokens length: 443302
ids length: 55212
compression ratio: 8.03X


## Decode

In [ ]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
  # given ids (list of integers), return Python string
  tokens = b"".join(vocab[idx] for idx in ids)
  text = tokens.decode("utf-8", errors="replace")
  return text

print(decode([325]))

স


In [ ]:
print(decode([1181])), max(ids)

�পনাদের �


(None, 1182)

## Encode

In [ ]:
def encode(text):
  # given a string, return list of integers (the tokens)
  tokens = list(text.encode("utf-8"))
  while len(tokens) >= 2:
    stats = get_stats(tokens)
    pair = min(stats, key=lambda p: merges.get(p, float("inf")))
    if pair not in merges:
      break # nothing else can be merged
    idx = merges[pair]
    tokens = merge(tokens, pair, idx)
  return tokens

print(encode("স"))

[325]


In [ ]:
print(encode("য"))

[256, 175]


In [ ]:
## Test whether encoding and then decoding of training text results into same text
text2 = decode(encode(ben_text))
print(text2 == ben_text)

True


In [ ]:
valtext = "Many common characters, including numerals, punctuation, and other symbols, are unified within the standard and are not treated as specific to any given writing system. Unicode encodes thousands of emoji, with the continued development thereof conducted by the Consortium as a part of the standard.[4] Moreover, the widespread adoption of Unicode was in large part responsible for the initial popularization of emoji outside of Japan. Unicode is ultimately capable of encoding more than 1.1 million characters."
valtext2 = decode(encode(valtext))
print(valtext2 == valtext)

True


### Forced splits using regex patterns


In [ ]:
import regex as re
# gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")
# https://www.regular-expressions.info/refext.html
# https://stackoverflow.com/questions/70985982/how-to-check-if-a-python-string-is-a-valid-bengali-word-using-regular-expression
ben_pat = re.compile(r""" ?\p{Bengali}+| ?[^\s\p{Bengali}]+|\s+(?!\S)|\s+""")

print(re.findall(ben_pat, """সবাই যা করতে চায় তা করতে চায়নি।
একা থাকার একটা অসুবিধে হোলো যে কথা বলার মতো কেউ থাকে না।
আপনার সমস্ত অর্থ দিয়ে আপনি যা চান তা কিনতে সক্ষম হওয়া উচিত।
বাইরে এতই গরম যে আমি পুরোদিন আমার শীততাপ নিয়ন্ত্রিত বাড়িতে থাকতে চাই।
সকালে সূর্য পূর্ব দিকে ওঠে এবং সন্ধ্যায় পশ্চিমে অস্ত যায়।
আমি শুনলাম যে তাঁরা পার্ক স্ট্রিটের একটা বাড়ির ভিতের মধ্যে থেকে একটা কঙ্কাল পেয়েছেন।
টম যখন শুনলো যে মেরি আরে জন বিয়ে করেছে তখন তাকে দেখে বেশ অবাক মনে হয়েছিলো।
আমার মনে হয় টম যে সোনা পেয়েছে সেটা তার কাছে রাখতে দেওয়া হবে এমন সম্ভাবনা খুব কম।
টম মেরিকে বললো যে ও নিজেকে হত্যা করতে চলেছিলো, কিন্ত তা করার মতো সাহস ছিলো না।
টমের সঙ্গে কাজ করা খুব বিরক্তিকর কারণ ও কখনই মেনে নেয় না যে ও ভুল করেছে।
আমি ভেবেছিলাম এটা করা সহজ হবে, কিন্তু আমরা সারাদিন ধরে কাজ করেছি আর এখনো শেষ করে উঠতে পারিনি।
আমি ভেবেছিলাম এটা করা সহজ হবে, কিন্তু আমরা সারাদিন ধরে কাজ করেছি আর এখনো শেষ করে উঠতে পারিনি।
বছরের বারোটা মাস হলো জানুয়ারি, ফেব্রুয়ারি, মার্চ, এপ্রিল, মে, জুন জুলাই, আগস্ট, সেপ্টেম্বর, অক্টোবর, নভেম্বর আর ডিসেম্বর।"""))

['সবাই', ' যা', ' করতে', ' চায়', ' তা', ' করতে', ' চায়নি', '।', '\n', 'একা', ' থাকার', ' একটা', ' অসুবিধে', ' হোলো', ' যে', ' কথা', ' বলার', ' মতো', ' কেউ', ' থাকে', ' না', '।', '\n', 'আপনার', ' সমস্ত', ' অর্থ', ' দিয়ে', ' আপনি', ' যা', ' চান', ' তা', ' কিনতে', ' সক্ষম', ' হওয়া', ' উচিত', '।', '\n', 'বাইরে', ' এতই', ' গরম', ' যে', ' আমি', ' পুরোদিন', ' আমার', ' শীততাপ', ' নিয়ন্ত্রিত', ' বাড়িতে', ' থাকতে', ' চাই', '।', '\n', 'সকালে', ' সূর্য', ' পূর্ব', ' দিকে', ' ওঠে', ' এবং', ' সন্ধ্যায়', ' পশ্চিমে', ' অস্ত', ' যায়', '।', '\n', 'আমি', ' শুনলাম', ' যে', ' তাঁরা', ' পার্ক', ' স্ট্রিটের', ' একটা', ' বাড়ির', ' ভিতের', ' মধ্যে', ' থেকে', ' একটা', ' কঙ্কাল', ' পেয়েছেন', '।', '\n', 'টম', ' যখন', ' শুনলো', ' যে', ' মেরি', ' আরে', ' জন', ' বিয়ে', ' করেছে', ' তখন', ' তাকে', ' দেখে', ' বেশ', ' অবাক', ' মনে', ' হয়েছিলো', '।', '\n', 'আমার', ' মনে', ' হয়', ' টম', ' যে', ' সোনা', ' পেয়েছে', ' সেটা', ' তার', ' কাছে', ' রাখতে', ' দেওয়া', ' হবে', ' এমন', ' সম্ভাবনা', ' খুব', ' কম', '।', 

## Bengali tokenizer with regex

In [23]:
import regex as re

In [24]:
vocab_size = 5001 # the desired final vocabulary size
num_merges = vocab_size - 256

In [25]:
# regex pattern for bengali text
regex_pat = re.compile(r""" ?\p{Bengali}+| ?[^\s\p{Bengali}]+|\s+(?!\S)|\s+""")

In [26]:
# split the text up into text chunks
text_chunks = re.findall(regex_pat, ben_text)

In [27]:
# input text preprocessing
ids = [list(ch.encode("utf-8")) for ch in text_chunks]

In [28]:
def get_stats(ids, counts= None):
    counts = {} if counts is None else counts
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
    newids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

In [29]:
# iteratively merge the most common pairs to create new tokens
merges = {} # (int, int) -> int
vocab = {idx: bytes([idx]) for idx in range(256)} # idx -> bytes
for i in range(num_merges):
    # count the number of times every consecutive pair appears
    stats = {}
    for chunk_ids in ids:
        # passing in stats will update it in place, adding up counts
        get_stats(chunk_ids, stats)
    # find the pair with the highest count
    pair = max(stats, key=stats.get)
    if stats[pair] < 2: # there should be at least 2 occurances
        print("exitting - less than 2 occurances")
        break
    # mint a new token: assign it the next available id
    idx = 256 + i
    # replace all occurrences of pair in ids with idx
    ids = [merge(chunk_ids, pair, idx) for chunk_ids in ids]
    # save the merge
    merges[pair] = idx
    vocab[idx] = vocab[pair[0]] + vocab[pair[1]]


print(f"vocab size: {len(vocab)}")
print(f"Number of merges: {i}")
print(f"Number of occurences of last merged tokens: {stats[pair]}")

exitting - less than 2 occurances
vocab size: 4734
Number of merges: 4478
Number of occurences of last merged tokens: 1


In [30]:
print("tokens length:", len(tokens))
print("ids length:", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

tokens length: 443302
ids length: 43603
compression ratio: 10.17X


## Decode

In [31]:
def decode(ids):
    # given ids (list of integers), return Python string
    part_bytes = []
    for idx in ids:
        if idx in vocab:
            part_bytes.append(vocab[idx])
        else:
            raise ValueError(f"invalid token id: {idx}")
    text_bytes = b"".join(part_bytes)
    text = text_bytes.decode("utf-8", errors="replace")
    return text

print(decode([325]))

াল


In [32]:
print(decode([999]))

আজ


## Encode

In [33]:
def _encode_chunk(text_bytes):
        # return the token ids
        # let's begin. first, convert all bytes to integers in range 0..255
        ids = list(text_bytes)
        while len(ids) >= 2:
            # find the pair with the lowest merge index
            stats = get_stats(ids)
            pair = min(stats, key=lambda p: merges.get(p, float("inf")))
            # subtle: if there are no more merges available, the key will
            # result in an inf for every single pair, and the min will be
            # just the first pair in the list, arbitrarily
            # we can detect this terminating case by a membership check
            if pair not in merges:
                break # nothing else can be merged anymore
            # otherwise let's merge the best pair (lowest merge index)
            idx = merges[pair]
            ids = merge(ids, pair, idx)
        return ids

def encode(text):
    # split text into chunks of text by categories defined in regex pattern
    text_chunks = re.findall(regex_pat, text)
    # all chunks of text are encoded separately, then results are joined
    ids = []
    for chunk in text_chunks:
        chunk_bytes = chunk.encode("utf-8") # raw bytes
        chunk_ids = _encode_chunk(chunk_bytes)
        ids.extend(chunk_ids)
    return ids


print(encode("স"))

[295]


In [34]:
print(encode("বিরক্তিকর"))

[285, 453, 790, 2697]


In [35]:
print([decode([i]) for i in encode("বিরক্তিকর")])

['ব', 'ির', 'ক্ত', 'িকর']


In [36]:
## Test whether encoding and then decoding of training text results into same text
text2 = decode(encode(ben_text))
print(text2 == ben_text)

True


In [37]:
valtext = """
বিশেষভাবে হত্যাকা-ের শিকার হচ্ছে শিশুরা।
অন্যদের কোনওরকমভাবে হয়রান করা হচ্ছে না।
বিক্ষোভকারীদের ওপর পুলিশ কাঁদানে গ্যাস ছোড়ে।
সেটা ফেরত পাওয়া খুব সোজা ব্যাপার নয়, অনিমেষ।
পরে হাসপাতাল থেকেও তাঁকে বের করে দেওয়া হয়।
নয়তো কার?
সম্প্রতি শারীরিক অসুস্থতার কারণে তাঁকে ভর্তি করা হয় সেখানকার হাসপাতালে।
ইতিমধ্যে ছবির শুটিংও শুরু হয়ে গেছে মুম্বইয়ের ধারাবি অঞ্চলে।
"""
valtext2 = decode(encode(valtext))
print(valtext2 == valtext)

True


### Save the vocab, merges and the regex

In [38]:
import pickle

In [39]:
save_dict = {"vocab": vocab, "merges": merges, "regex_pat": regex_pat}
with open("bengali_tokenizer.pkl", "wb") as f:
    pickle.dump(save_dict, f)

In [40]:
import os
os.makedirs('to_upload', exist_ok=True)

In [41]:
!cp bengali_tokenizer.pkl to_upload/

In [42]:
from huggingface_hub import HfApi
from google.colab import userdata
userdata.get('HF_TOKEN')
api = HfApi()

In [43]:
api.upload_folder(
    folder_path="./to_upload",
    repo_id="sayanbanerjee32/bengali_tokenizer",
    repo_type="model",
)

bengali_tokenizer.pkl:   0%|          | 0.00/141k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sayanbanerjee32/bengali_tokenizer/commit/558bff5e6f36482c8714e406d60e8dbdbf335b63', commit_message='Upload folder using huggingface_hub', commit_description='', oid='558bff5e6f36482c8714e406d60e8dbdbf335b63', pr_url=None, pr_revision=None, pr_num=None)